In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

In [5]:
# 定义简单的卷积神经网络
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # 卷积层：输入通道数为1（灰度图像），输出通道数为10，核大小为5x5
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        # 卷积层：输入通道数为10，输出通道数为20，核大小为5x5
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()  # Dropout 层用于防止过拟合
        # 全连接层：输入特征数为320，输出特征数为50
        self.fc1 = nn.Linear(320, 50)
        # 输出层：输入特征数为50，输出特征数为10（MNIST数据集有10个类别）
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        # 第一层卷积 + 最大池化 + ReLU激活
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # 第二层卷积 + Dropout + 最大池化 + ReLU激活
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        # 将多维张量展平成一维
        x = x.view(-1, 320)
        # 全连接层 + ReLU激活 + Dropout
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        # 输出层
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
# 设置设备为CPU
device = torch.device("cpu")

# 加载MNIST数据集
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=False)

# 初始化网络并移动到CPU
model = SimpleCNN().to(device)

# 定义优化器
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw



In [7]:
# 训练函数
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [8]:
# 测试函数
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [9]:
# 开始训练和测试
for epoch in range(1, 4):  # 这里设置为训练3个epochs
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.259072
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.301721
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.298400
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.261734
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.195748
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.193276
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.167448
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.076039
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.022713
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.843124
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.867559
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.728183
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.711453
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.600980
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.385606
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.479333
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.375232
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.162003
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.267240
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.273263
Train Epoch: 1 [12800/60000 (